<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Cooler-Python-API" data-toc-modified-id="Cooler-Python-API-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Cooler Python API</a></span><ul class="toc-item"><li><span><a href="#Direct-access-with-h5py" data-toc-modified-id="Direct-access-with-h5py-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Direct access with <code>h5py</code></a></span></li><li><span><a href="#The-Cooler-class" data-toc-modified-id="The-Cooler-class-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>The <code>Cooler</code> class</a></span><ul class="toc-item"><li><span><a href="#The-info-dictionary" data-toc-modified-id="The-info-dictionary-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>The info dictionary</a></span></li><li><span><a href="#Table-Views" data-toc-modified-id="Table-Views-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Table Views</a></span></li><li><span><a href="#Bin-annotation" data-toc-modified-id="Bin-annotation-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Bin annotation</a></span></li><li><span><a href="#Enter-The-Matrix" data-toc-modified-id="Enter-The-Matrix-1.2.4"><span class="toc-item-num">1.2.4&nbsp;&nbsp;</span>Enter The Matrix</a></span></li><li><span><a href="#Balancing-your-selection" data-toc-modified-id="Balancing-your-selection-1.2.5"><span class="toc-item-num">1.2.5&nbsp;&nbsp;</span>Balancing your selection</a></span></li><li><span><a href="#Genomic-coordinate-range-selection" data-toc-modified-id="Genomic-coordinate-range-selection-1.2.6"><span class="toc-item-num">1.2.6&nbsp;&nbsp;</span>Genomic coordinate range selection</a></span></li></ul></li></ul></li></ul></div>

# Cooler Python API

Pre-requisites:

- Basic Python knowledge
- Some experience with the NumPy array package (or MATLAB)

This walkthrough also makes use of:

- Jupyter (IPython) Notebook. That's what you're using right now!
- Pandas, the dataframe package (similar to R data.frames)
- h5py, the package to interact with HDF5 files from Python
- matplotlib, the MATLAB-inspired plotting package for Python

To navigate this notebook:

- Click on a code cell and execute its code by pressing `shift+enter` or clicking the "play" button on the toolbar.
- While the code cell is running the prompt on the left will look like `In [*]:` and will display the execution count when it is done.
- Execution output will be displayed beneath each cell.
- To restart the notebook, use the options in the `Kernel` dropdown menu.
- You can also run the entire notebook in one go with the `Restart & Run All` option

In [ ]:
# Import the packages we will use
import os.path as op
import matplotlib.pyplot as plt
import numpy as np
import pandas
import h5py

import cooler

In [ ]:
# The following directive activates inline plotting
%matplotlib inline

In [ ]:
filepath = 'data/Rao2014-GM12878-MboI-allreps-filtered.5kb.cool'

# Download a high resoltion COOL file from the interwebs (this will take a few...)
# The ! at the beginning of the line tells IPython to run the line in the shell.
if not op.exists(filepath):
    !wget ftp://cooler.csail.mit.edu/coolers/hg19/Rao2014-GM12878-MboI-allreps-filtered.5kb.cool -O {filepath}

## Direct access with `h5py`

The `h5py` library (HDF5 for Python) provides an excellent Pythonic interface between HDF5 and native [NumPy](http://www.numpy.org/) arrays and dtypes. It allows you to treat an HDF5 file like a dictionary with complete access to the file's contents as well as the ability to manipulate groups and read or write datasets and attributes. There is additionally a low-level API that wraps the `libhdf5` C functions directly. See the [h5py docs](http://docs.h5py.org/en/latest/index.html).

In [ ]:
h5 = h5py.File(filepath, 'r')

In [ ]:
h5

In [ ]:
h5.keys()

Files and Groups are `dict`-like.

In [ ]:
h5['pixels']

In [ ]:
list(h5['pixels'].keys())

`h5py` dataset objects are **views** onto the data on disk

In [ ]:
h5['pixels']['bin2_id']

Slicing or indexing returns a numpy array in memory.

In [ ]:
h5['pixels']['bin2_id'][:10]

In [ ]:
h5['pixels']['count'][:10]

In [ ]:
h5.close()

The Python `cooler` package is just a thin wrapper over `h5py`.

- It lets you access the data tables as [Pandas](http://pandas.pydata.org/) [data frames and series](http://pandas.pydata.org/pandas-docs/stable/10min.html). 
- It also provides a _matrix abstraction_: letting you query the upper triangle pixel table as if it were a full rectangular [sparse matrix](http://www.scipy-lectures.org/advanced/scipy_sparse/storage_schemes.html) via [SciPy](http://www.scipy-lectures.org/index.html).

See below.

## The `Cooler` class

Accepts a file path or an open HDF5 file object.

NOTE: Using a filepath allows the `Cooler` object to be serialized/pickled since the file is only opened when needed.


In [ ]:
c = cooler.Cooler(filepath)

### The info dictionary

In [ ]:
c.info

### Table Views
Tables are accessed via methods.

In [ ]:
c.chroms()

The return value is a selector or "view" on a table that accepts column and range queries ("slices").

- Column selections return a new view.
- Range selections return pandas [DataFrames or Series](http://pandas.pydata.org/pandas-docs/stable/dsintro.html).

In [ ]:
c.chroms()[1:5]

In [ ]:
# get the whole table
c.chroms()[:]

In [ ]:
# more convenient access to chromosomes
c.chromnames

In the bin table, the **weight** column contains the _matrix balancing weights_ computed for each genomic bin.

In [ ]:
# more convenient access to chromosome lengths
c.chromsizes

In [ ]:
c.bins()[:10]

Selecting a list of columns returns a new DataFrame view on that subset of columns

In [ ]:
bins = c.bins()[['chrom', 'start', 'end']]
bins

In [ ]:
bins[:10]

Selecting a single column returns a Series view

In [ ]:
weights = c.bins()['weight']
weights

In [ ]:
weights[500:510]

The pixel table contains the non-zero upper triangle entries of the contact map.

In [ ]:
c.pixels()[:10]

Use the `join=True` option if you would like to expand the bin IDs into genomic bin coordinates by joining the output with the bin table.

In [ ]:
c.pixels(join=True)[:10]

Pandas lets you readily dump any table selection to tabular text file.

In [ ]:
df = c.pixels(join=True)[:100]

# tab-delimited file, don't write the index column or header row
df.to_csv('data/myselection.txt', sep='\t', index=False, header=False)

In [ ]:
!head data/myselection.txt

### Bin annotation

Another way to annotate the bins in a data frame of pixels is to use `cooler.annotate`. It does a [left outer join](http://chris.friedline.net/2015-12-15-rutgers/lessons/python2/04-merging-data.html) from the `bin1_id` and `bin2_id` columns onto a data frame indexed by bin ID that describes the bins.

In [ ]:
bins = c.bins()[:]  # fetch all the bins

pix = c.pixels()[100:110]  # select some pixels with unannotated bins
pix

In [ ]:
cooler.annotate(pix, bins)

In [ ]:
cooler.annotate(pix, bins[['weight']], replace=False)

### Enter The Matrix

Finally, the `matrix` method provides a 2D-sliceable view on the data. It allows you to query the data on file as a full rectangular contact matrix.

In [ ]:
c.matrix()

The result of a query is a 2D NumPy array.

In [ ]:
arr = c.matrix(balance=False)[1000:1200, 1000:1200]
arr

Use **`sparse=True`** to return `scipy.sparse.coo_matrix` objects instead.

In [ ]:
mat = c.matrix(balance=False, sparse=True)[1000:1200, 1000:1200]
mat

It is straightforward to convert to a dense 2D numpy array.

In [ ]:
arr = mat.toarray()
arr

Notice that the lower triangle has been automatically filled in.

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
im = ax.matshow(np.log10(arr), cmap='YlOrRd')
fig.colorbar(im)

Notice the light and dark "banded" appearance? That's because you are looking at the unnormalized counts.

### Balancing your selection

We usually normalize or "correct" Hi-C using a technique called matrix balancing. This involves finding a set of weights or biases $b_i$ for each bin $i$ such that

$$ Normalized[i,j] = Observed[i,j] \cdot b[i] \cdot b[j], $$

such that the marginals (i.e., row/column sums) of the global contact matrix are flat and equal.

Cooler can store the pre-computed balancing weights in the bin table.

Here's one way to manually apply them to balance your selection.

In [ ]:
# get the balancing weights as a numpy array
weights = c.bins()['weight']  # view
bias = weights[1000:1200]     # series
bias = bias.values            # array

# fetch a sparse matrix
mat = c.matrix(balance=False, sparse=True)[1000:1200, 1000:1200]

# apply the balancing weights
mat.data = bias[mat.row] * bias[mat.col] * mat.data

# convert to dense numpy array
arr = mat.toarray()

As a shortcut, we get the same result by passing **`balance=True`** to the matrix view constructor.

In [ ]:
arr2 = c.matrix(balance=True, sparse=True)[1000:1200, 1000:1200].toarray()
np.allclose(arr, arr2, equal_nan=True)

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
im = ax.matshow(np.log10(arr), cmap='YlOrRd')
fig.colorbar(im)

### Genomic coordinate range selection

The bin table, pixel table and matrix views also accept UCSC-style genomic range strings or (chrom, start, end) triples.

In [ ]:
c.bins().fetch('chr2:10,000,000-20,000,000')

In [ ]:
cis = c.matrix(sparse=True).fetch('chr21')
cis.shape

In [ ]:
trans = c.matrix(sparse=True).fetch('chr21', 'chr22')
trans.shape